## Notebook to create the splitted data into chunks, top1 and top3

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re


1. check overlapping - done
2. create train+test / dev datasets - done
3. score datasets with base model (pretrained models)
    qnli-electra-base  
4. Try with chunks without overlapping; try with token split half half; find max number of tokens for a query and keep the rest for sentence (max number)
    
    
    compare score, roc-auc, accuracy (?) choose 1
    
5. save to .csv 
6. have fun :)
    1. difference bw scores and label, create dataset with 1 row for each pair, train a model 
    2. 0.5 (JOINED) difference bw scores and label, create dataset with 1 row for each pair, train a model
    3. compare models 1 and 2, select the best one 
    4. tune parameters for the best model
7. compare results for different dataset sizes 

In [ ]:
dev_data = 'dev_data.csv'
all_docs = 'all_docs.csv'
dev_queries = 'dev_queries.csv'
test_data = 'test_data.csv'
training_data = 'training_data.csv'
lucene_data = 'raw_dev_Lucene_retrievals.csv'

# pickled train and test files
train = 'train_chunks.csv'
test = 'test_chunks.csv'

split_data = 'split_text_256.csv'
encoded_data = 'encoded.csv'

In [ ]:
df_train = pd.read_csv(train)
df_test = pd.read_csv(test)

# df_split_text = pd.read_csv(dev_data)

In [ ]:
model_checkpoint = "cross-encoder/stsb-TinyBERT-L-4"
batch_size = 8

In [ ]:
df_train.head()
#df_split_text

In [ ]:
df_test.head()

In [ ]:
sentence1_key = 'doc_text'
sentence2_key = 'Query'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
df_train['label'] = df_train['label'].astype(np.float64)
df_test['label'] = df_test['label'].astype(np.float64)

df_train['doc_text'] = df_train['doc_text'].astype(str)
df_test['doc_text'] = df_test['doc_text'].astype(str)

df_train['Query'] = df_train['Query'].astype(str)
df_test['Query'] = df_test['Query'].astype(str)
# df_split_text['label'] = df_split_text['label'].astype(np.float64)

## Top records

In [ ]:
model = CrossEncoder(model_checkpoint)

In [ ]:
scores_train = model.predict(df_train[['Query', 'doc_text']].values.tolist(), show_progress_bar=True)

In [ ]:
scores_test = model.predict(df_test[['Query', 'doc_text']].values.tolist(), show_progress_bar=True)

In [ ]:
print("one's share train: ", df_train['label'].mean())
print("one's share test: ", df_test['label'].mean())

In [ ]:
# df_train = df_train[["doc_text", "Query", "label"]]
# df_test = df_test[["doc_text", "Query", "label"]]

# df_val = df_val[["doc_text", "query", "label"]]
hf_train = datasets.Dataset.from_pandas(df_train)
hf_test = datasets.Dataset.from_pandas(df_test)
# hf_val = datasets.Dataset.from_pandas(df_val)

In [ ]:
hf = datasets.DatasetDict({"train": hf_train,
                           "test": hf_test})
                           #"val": hf_val})

In [ ]:
hf

In [ ]:
def preprocess_function(examples):
    return tokenizer(str(examples[sentence1_key]), str(examples[sentence2_key]), truncation=True)

In [ ]:
encoded_hf = hf.map(preprocess_function, remove_columns=("doc_text", "Query"), batch_size = 512)

In [ ]:
model = CrossEncoder(model_checkpoint)

In [ ]:
scores_train = model.predict(df_train[['Query', 'doc_text']].values.tolist(), show_progress_bar=True)

In [ ]:
scores_test = model.predict(df_test[['Query', 'doc_text']].values.tolist(), show_progress_bar=True)

In [ ]:
df_train['score'] = scores_train

In [ ]:
df_test['score'] = scores_test

In [ ]:
df_train['difference'] = abs(df_train['label'] - df_train['score'])
df_test['difference'] = abs(df_test['label'] - df_test['score'])

In [ ]:
df_train_3 = pd.DataFrame(df_train.sort_values('difference').groupby(['doc_number', 'query_number']).head(3)).reset_index()
df_test_3 = pd.DataFrame(df_test.sort_values('difference').groupby(['doc_number', 'query_number']).head(3)).reset_index()

In [ ]:
df_train_1 = pd.DataFrame(df_train.sort_values('difference').groupby(['doc_number', 'query_number']).head(1)).reset_index()
df_test_1 = pd.DataFrame(df_test.sort_values('difference').groupby(['doc_number', 'query_number']).head(1)).reset_index()

In [ ]:
df_train[(df_train['query_number'] == 166043) & (df_train['doc_number'] == 31135)]

In [ ]:
df_train_3[(df_train_3['query_number'] == 166043) & (df_train_3['doc_number'] == 31135)]

In [ ]:
df_train_1[(df_train_1['query_number'] == 166043) & (df_train_1['doc_number'] == 31135)]

In [ ]:
print(df_train_1.shape)
print(df_train_3.shape)

In [ ]:
df_train_1.to_csv("train_top1.csv")
df_test_1.to_csv("test_top1.csv")

In [ ]:
df_train_3.to_csv("train_top3.csv")
df_test_3.to_csv("test_top3.csv")

In [ ]:
print("one's share train: ", df_train_1['label'].mean())
print("one's share test: ", df_test_1['label'].mean())

In [ ]:
print("one's share train: ", df_train_3['label'].mean())
print("one's share test: ", df_test_3['label'].mean())

In [ ]:
# from sklearn.model_selection import train_test_split

# df_train, _ = train_test_split(df_train, test_size=0.8, random_state=42, shuffle=True)
# df_test, _ = train_test_split(df_test, test_size=0.2, random_state=42)